# Ground Motion Displacement RMS vs Time

*an example simple tutorial for getting seismic data, computing the power spectral densities, extracting the RMS and plotting*

Required:

- python
- obspy (and its dependencies)
- pandas
- jupyter
- notebook
- tqdm

this should be easy to set up in a conda env: ``conda create -c conda-forge -n covid python=3.7 obspy pandas jupyter notebook tqdm``

Author: Thomas Lecocq @seismotom, Fred Massin @fmassin, Claudio Satriano @claudiodsf

## Step 1: imports

In [49]:
import datetime
import os
from glob import glob

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # to edit text in Illustrator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
import numpy as np
import pandas as pd
import tqdm
import warnings

from obspy import UTCDateTime, read, read_inventory
from obspy.clients.filesystem.sds import Client

from obspy.signal import PPSD

import seismosocialdistancing

In [50]:
from importlib import reload
reload(seismosocialdistancing)

<module 'seismosocialdistancing' from '/home/dariaorogom/Sismologia/seismosocialdistancing.py'>

## Step 2: Define Start/End dates and Seismic Channel

You'll have to make sure the seed_id you request is indeed available from the ``data_provider``

In [51]:
start = UTCDateTime("2022-12-01")
# Leaving UTCDateTime() empty means "now":
end = UTCDateTime("2023-1-31")

network = "AM"
station = "R95F0"
location = "00"
channel = "HDF"
dataset = "CGEO"
time_zone = 'America/Mexico_City'
sitedesc = "Querétaro (Querétaro, MX)"

data_provider = "/home/dariaorogom/Desktop/Infra"
logo = None # 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/Logo_SED_2014.png/220px-Logo_SED_2014.png'
#bans = {"2020-03-15 00:00":'Restaurants/Bars/Schools closed', 
#        "2020-03-18 12:00":'Non-essential shops closed'}


## Step 3: Download the seismic waveform data

This step is coded so that only the last day is redownloaded if the daily files are present on the disk.

The request gets the target day +- 30 minutes to avoid having gaps at the end of each day (need 1 window covering midnight).

In [52]:
datelist = pd.date_range(start.datetime, min(end, UTCDateTime()).datetime, freq="D")
c = Client(data_provider)

nslc = "{}.{}.{}.{}".format(network, station, location, channel)
# make sure that wildcard characters are not in nslc
nslc = nslc.replace("*", "").replace("?", "")
pbar = tqdm.tqdm(datelist)
for day in pbar:
    datestr = day.strftime("%Y-%m-%d")
    fn = "{}_{}_{}.mseed".format(dataset, datestr, nslc)
    if day != UTCDateTime().datetime and os.path.isfile(fn):
        continue
    else:
        pbar.set_description("Fetching %s" % fn)
        st = c.get_waveforms(network, station, location, channel,
                             UTCDateTime(day)-1801, UTCDateTime(day)+86400+1801, attach_response=True)
        #st.write(fn)
resp = read_inventory("Infra.xml")
print(resp)


Fetching CGEO_2023-01-31_AM.R95F0.00.HDF.mseed: 100%|█| 62/62 [01:20<00:00,  1.3


Inventory created at 2023-07-04T22:11:40.245489Z
	Sending institution: scxml import (ObsPy Inventory)
	Contains:
		Networks (1):
			AM
		Stations (8):
			AM.R6417 (Casa-Juan)
			AM.R7E22 (Casa-Adolfo)
			AM.R8358 (Casa)
			AM.R95F0 (Casa-Alex)
			AM.RD07C (Montegrande-Colima)
			AM.RDBC3 (Oficina)
			AM.RE7AC (Casa Liliana)
			AM.REDAC (FACIMAR)
		Channels (8):
			AM.R6417.00.HDF, AM.R7E22.00.HDF, AM.R8358.00.HDF, AM.R95F0.00.HDF
			AM.RD07C.00.HDF, AM.RDBC3.00.HDF, AM.RE7AC.00.HDF, 
			AM.REDAC.00.HDF


In [53]:
#resp.write("RD07C.xml",format="STATIONXML")

In [54]:
meta = {'sensitivity':resp[0][0][0].response.instrument_sensitivity.value}

In [55]:
meta

{'sensitivity': 4000.0}

In [56]:
1/0.1

10.0

## Step 4: Compute PPSDs using custom parameters

These parameters are set to allow the PSDs to be "nervous", not as smooth as the default PQLX ones.

In [57]:
force_reprocess = False
pbar = tqdm.tqdm(datelist)
for day in pbar:
    datestr = day.strftime("%Y-%m-%d")
    fn_in = "{}_{}_{}.mseed".format(dataset, datestr, nslc)
    pbar.set_description("Processing %s" % fn_in)
    stall = read(fn_in, headonly=True)
    for mseedid in list(set([tr.id for tr in stall])):
        fn_out = "{}_{}_{}.npz".format(dataset, datestr, mseedid)
        if os.path.isfile(fn_out) and not force_reprocess:
            continue
        st = read(fn_in, sourcename=mseedid)
        st.attach_response(resp)
        ppsd = PPSD(st[0].stats, metadata=resp,
                    ppsd_length=1800, overlap=0.5,
                    period_smoothing_width_octaves=0.025,
                    period_step_octaves=0.0125,
                    period_limits=(0.02, 1),
                    db_bins=(-200, 20, 0.25),
                    special_handling='infrasound')
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ppsd.add(st)
        ppsd.save_npz(fn_out[:-4])
        del st, ppsd
    del stall


Processing CGEO_2022-12-01_AM.R95F0.00.HDF.mseed:   0%|  | 0/62 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'CGEO_2022-12-01_AM.R95F0.00.HDF.mseed'

## Step 5: Reload daily PSDs from the disk and create a single PPSD object:

In [ ]:
ppsds = {}
pbar = tqdm.tqdm(datelist)
for day in pbar:
    datestr = day.strftime("%Y-%m-%d")
    fn_pattern = "{}_{}_*.npz".format(dataset, datestr)
    pbar.set_description("Reading %s" % fn_pattern)
    for fn in glob(fn_pattern):
        mseedid = fn.replace(".npz", "").split("_")[-1]
        if mseedid not in ppsds:
            ppsds[mseedid] = PPSD.load_npz(fn)#, allow_pickle=True)
        else:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                ppsds[mseedid].add_npz(fn)#, allow_pickle=True)

## Step 6: Standard plots:

In [ ]:
#[ppsd.plot(max_percentage=10,show_noise_models=True) for mseedid, ppsd in ppsds.items()]
[ppsd.plot_temporal([0.2,0.1,1/20,1/30]) for mseedid, ppsd in ppsds.items()]
#[ppsd.plot_spectrogram(clim=(-100,0)) for mseedid, ppsd in ppsds.items()]
plt.ylim(0,-100)

In [ ]:
[ppsd.plot(max_percentage=10,show_noise_models=True,period_lim=(0.02, 1),show_mean=True) for mseedid, ppsd in ppsds.items()]
#[ppsd.plot(max_percentage=10,show_noise_models=False,xaxis_frequency=True) for mseedid, ppsd in ppsds.items()]

## Step 7: Process PSDs to extract the RMS(displacement)

This can be done for multiple filters at once (``freqs`` below):

In [ ]:
def dfrms(a):
    return np.sqrt(np.trapz(a.values, a.index))

def df_rms(d, freqs, output="VEL"):
    d = d.dropna(axis=1, how='all')
    RMS = {}
    for fmin, fmax in freqs:
        
        ix = np.where((d.columns>=fmin) & (d.columns<=fmax))[0]
        spec = d.iloc[:,ix]
        f = d.columns[ix]
        
        w2f = (2.0 * np.pi * f)

        # The acceleration power spectrum (dB to Power! = divide by 10 and not 20!)
        amp = 10.0**(spec/10.) 
        if output == "ACC":
            RMS["%.1f-%.1f"%(fmin, fmax)] = amp.apply(dfrms, axis=1)
            continue
        
        # velocity power spectrum (divide by omega**2)
        vamp = amp / w2f**2
        if output == "VEL":
            RMS["%.1f-%.1f"%(fmin, fmax)] = vamp.apply(dfrms, axis=1)
            continue
                
        # displacement power spectrum (divide by omega**2)
        damp = vamp / w2f**2
       
        RMS["%.1f-%.1f"%(fmin, fmax)] = damp.apply(dfrms, axis=1)

    return pd.DataFrame(RMS, index=d.index)#.tz_localize("UTC")#.dropna()


In [ ]:
# Define frequency bands of interest:
freqs = [(0.1,1.0),(1.0,20.0),(4.0,14.0),(20.0,40.0)]

displacement_RMS = {}
for mseedid, ppsd in tqdm.tqdm(ppsds.items()):
    ind_times = pd.DatetimeIndex([d.datetime for d in ppsd.current_times_used])
    data = pd.DataFrame(ppsd.psd_values, index=ind_times, columns=1./ppsd.period_bin_centers)
    data = data.sort_index(axis=1)
    displacement_RMS[mseedid] = df_rms(data, freqs, output="ACC")
    displacement_RMS[mseedid].to_csv("%s.csv" % mseedid)

In [ ]:
from importlib import reload
reload(seismosocialdistancing)

## Step 8: Custom plot for a single frequency band:

## Weekday / Time of day Analysis

## Noise distribution over time of the day  

In [ ]:
data = displacement_RMS

In [ ]:
for channelcode in list(set([k[:-1] for k in displacement_RMS])):
    print(channelcode)

In [ ]:
data={}

In [ ]:
band="20.0-40.0"
for o in 'ZENF':
    print(o)

In [ ]:
data[channelcode[-2:]+o] = displacement_RMS[channelcode+o][band]
data

In [ ]:
main=channelcode[-2:]+o
main
data[main]

In [ ]:
def localize_tz_and_reindex(df, freq="15Min", time_zone = "Europe/Brussels"):
    return df.copy().tz_localize("UTC").dropna().tz_convert(time_zone).tz_localize(None).resample(freq).mean().to_frame()

In [ ]:
data[main] = localize_tz_and_reindex(data[main], "30Min", time_zone = time_zone)

In [ ]:
data[main]

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday']
# Just a bunch of helper functions
def stack_wday_time(df):
    """Takes a DateTimeIndex'ed DataFrame and returns the unstaked table: hours vs day name"""
    return df.groupby(level=(0,1)).median().unstack(level=-1).T.droplevel(0)[days]#*1e9

In [ ]:
postloc = data[main].loc[list(bans.keys())[0]:]

In [ ]:
postloc = postloc.set_index([postloc.index.day_name(), postloc.index.hour+postloc.index.minute/60.])

In [ ]:
postloc

In [ ]:
cmap = plt.get_cmap("tab20")

In [ ]:
stack_wday_time(postloc).plot(figsize=(14,8),cmap = cmap)
                
plt.title("Daily Noise Levels in %s" % (channelcode[:]+main[-1]))
plt.ylabel("Pa")
plt.xlabel("Hour of day (local time)")
plt.grid()
plt.xlim(0,23)

In [ ]:
def radial_hours(N):
    hours = np.deg2rad(np.linspace(0, 360, N-1, endpoint=False))
    hours = np.append(hours, hours[0])
    return hours
def clock24_plot_commons(ax):
    # Set the circumference labels
    ax.set_xticks(np.linspace(0, 2*np.pi, 24, endpoint=False))
    ax.set_xticklabels(["%i h"%i for i in range(24)], fontsize=8)
    #ax.set_yticklabels(["%i nm" % i for i in np.arange(0,100, 10)], fontsize=7)
    ax.yaxis.set_tick_params(labelsize=10)

    # Make the labels go clockwise
    ax.set_theta_direction(-1)
    ax.legend(loc='center left',bbox_to_anchor=(1, 0, 0.5, 1.8))
    #ax.legend(loc="center right')
    # Place 0 at the top
    ax.set_theta_offset(np.pi/2.0)
    plt.xlabel("Hour (local time)", fontsize=10)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(12,6))
ax = plt.subplot(polar=True)
_ = stack_wday_time(postloc).copy()
_.loc[len(_)+1] = _.iloc[0]
_.index = radial_hours(len(_))
_.plot(ax=ax)
#plt.title("Infrasound", fontsize=12)
clock24_plot_commons(ax)
plt.suptitle("Day/Hour Median Noise levels %s\nStation %s - [%s] Hz" % (sitedesc,channelcode[:]+main[-1],band), fontsize=12)
plt.savefig('R8358_Clock.png')